# Bienvenido al día 2 de la workshop de Nuclio
## 15-Enero-2025

---

Objetivos de la sesión:

En la segunda sesión de nuestro workshop vamos a construir un recomendador de animes basado en la técnica de **"Content Based Recommendation System"** desde cero.

Buscamos que los presentes:

In [1]:
import os
import pandas as pd

In [2]:
CWD = os.getcwd()

In [3]:
PATH_INPUT_FOLDER = os.path.join(CWD, "input")

In [4]:
PATH_INPUT_FOLDER

'/Users/nicolaepopescul/code/nuclio_charlas/input'

In [5]:
os.listdir(PATH_INPUT_FOLDER)

['cf_anime.parquet.gzip', 'cf_rating.parquet.gzip']

In [6]:
rating = pd.read_parquet(os.path.join(PATH_INPUT_FOLDER, 'cf_rating.parquet.gzip'))
anime = pd.read_parquet(os.path.join(PATH_INPUT_FOLDER, 'cf_anime.parquet.gzip'))

In [7]:
rating = rating[rating["rating"] != -1]

In [8]:
rating.sample(5)

,user_id,anime_id,rating
5417966,51194,376,9
2778420,26095,1023,7
945203,8583,30276,7
1837107,17807,9982,10
439238,4512,20931,2


In [9]:
anime = anime[["anime_id", "name", "genre"]]

In [10]:
anime.head()

,anime_id,name,genre
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural"
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S..."


In [11]:
genres = anime["genre"].str.split(", ", expand = True)

In [12]:
anime = pd.concat([anime, genres], axis = 1)

In [13]:
anime.head(3)

,anime_id,name,genre,0,1,2,3,4,5,6,7,8,9,10,11,12
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Drama,Romance,School,Supernatural,None,None,None,None,None,None,None,None,None
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,None,None,None,None,None,None
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",Action,Comedy,Historical,Parody,Samurai,Sci-Fi,Shounen,None,None,None,None,None,None


In [14]:
anime_genre = (
    anime[["anime_id"] + genres.columns.tolist()]
    .melt(id_vars = "anime_id")
    [["anime_id", "value"]]
    .dropna()
    .pivot_table(
        index = "anime_id",
        columns = "value",
        aggfunc = len,
        fill_value = 0
    )
)

In [15]:
anime_genre.head(3)

value,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
anime_id,,,,,,,,,,,,,,,,,,,,,
1,1,1,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
oc = [
    'user_id',
    'name',
    'rating',
    'Action',
    'Adventure',
    'Cars',
    'Comedy',
    'Dementia',
    'Demons',
    'Drama',
    'Ecchi',
    'Fantasy',
    'Game',
    'Harem',
    'Hentai',
    'Historical',
    'Horror',
    'Josei',
    'Kids',
    'Magic',
    'Martial Arts',
    'Mecha',
    'Military',
    'Music',
    'Mystery',
    'Parody',
    'Police',
    'Psychological',
    'Romance',
    'Samurai',
    'School',
    'Sci-Fi',
    'Seinen',
    'Shoujo',
    'Shoujo Ai',
    'Shounen',
    'Shounen Ai',
    'Slice of Life',
    'Space',
    'Sports',
    'Super Power',
    'Supernatural',
    'Thriller',
    'Vampire',
    'Yaoi',
    'Yuri'
]

In [17]:
X = pd.merge(
    left = rating,
    right = anime_genre.reset_index(),
    how = "inner",
    on = ["anime_id"]
)

In [18]:
X = X.merge(right = anime[["anime_id", "name"]], how = "inner", on = ["anime_id"])

In [19]:
X = X[oc]

In [20]:
X.sample(10)

,user_id,name,rating,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
709534,7904,"Gate: Jieitai Kanochi nite, Kaku Tatakaeri",8,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1475802,17614,Soul Eater,7,1,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
685593,7673,One Outs,9,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1591498,19055,Upotte!!,8,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3526405,41191,Magic Knight Rayearth,6,0,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4864950,56449,Re:␣Hamatora,8,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4710513,54466,Mobile Suit Gundam MS IGLOO: The Hidden One Ye...,7,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3932763,46401,Tengen Toppa Gurren Lagann,10,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6228063,72088,Kämpfer,2,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5822501,66876,Sensitive Pornograph,10,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [21]:
X.iloc[:, 3::] = X.iloc[:, 3::].multiply(X["rating"], axis = 0)

In [22]:
X.sample(10)

,user_id,name,rating,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
2490423,28412,Gosick,9,0,0,0,0,0,0,9,...,0,0,0,0,0,0,0,0,0,0
3715482,43892,Inu x Boku SS,6,0,0,0,6,0,0,0,...,0,0,0,0,0,6,0,0,0,0
6059327,69790,K,9,9,0,0,0,0,0,0,...,0,0,0,0,9,9,0,0,0,0
4270336,50099,Fullmetal Alchemist: The Conqueror of Shamballa,9,0,0,0,9,0,0,9,...,0,0,0,0,0,0,0,0,0,0
3961437,46925,Mononoke Hime,10,10,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5642795,64838,Ao no Exorcist Movie Special,8,0,0,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6156643,71156,Inazuma Eleven,5,0,0,0,0,0,0,0,...,0,0,0,5,5,0,0,0,0,0
4545008,52773,Noragami,8,8,8,0,0,0,0,0,...,0,0,0,0,0,8,0,0,0,0
2882889,32914,Log Horizon,9,9,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2265924,26166,Hanamonogatari,8,0,0,0,8,0,0,0,...,0,0,0,0,0,8,0,0,0,0


In [23]:
ponderated_score = (
    X
    .drop(columns = ["name", "rating"])
    .groupby(["user_id"])
    .sum()
    .pipe(lambda df: df.divide(df.sum(axis = 1), axis = 0))
)

In [24]:
ponderated_score

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.136364,0.045455,0.000000,0.090909,0.000000,0.090909,0.000000,0.136364,0.045455,0.045455,...,0.0,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.0,0.000000
2,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.133255,0.104363,0.000000,0.085790,0.000000,0.007665,0.068101,0.011498,0.116156,0.016509,...,0.0,0.011498,0.000000,0.025354,0.020637,0.053656,0.009434,0.001769,0.0,0.000000
5,0.085026,0.055432,0.002819,0.156665,0.000117,0.009278,0.047798,0.029477,0.049207,0.004815,...,0.0,0.038638,0.003406,0.027481,0.020082,0.050029,0.005989,0.003641,0.0,0.000000
7,0.081957,0.040153,0.000000,0.125587,0.002434,0.015123,0.032331,0.046237,0.054841,0.009995,...,0.0,0.031636,0.000000,0.007648,0.022771,0.050408,0.007996,0.005823,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73512,0.062731,0.031365,0.000000,0.084871,0.018450,0.018450,0.125461,0.000000,0.036900,0.000000,...,0.0,0.042435,0.000000,0.000000,0.000000,0.105166,0.000000,0.000000,0.0,0.000000
73513,0.074074,0.061728,0.000000,0.121693,0.007055,0.000000,0.109347,0.028219,0.063492,0.000000,...,0.0,0.030864,0.023810,0.006173,0.000000,0.044092,0.006173,0.006173,0.0,0.000000
73514,0.000000,0.166667,0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.166667,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


# Parte de Inferencia

In [25]:
random_user = pd.Series(rating["user_id"].unique()).sample().iloc[0]

In [26]:
random_user

63558

In [27]:
random_animes = anime_genre.reset_index()["anime_id"].sample(10).values.tolist()

In [28]:
random_animes

[30407, 29995, 17693, 6374, 20543, 20087, 6721, 27525, 9689, 342]

In [29]:
random_animes_genres = anime_genre[anime_genre.index.isin(random_animes)]

In [30]:
random_animes_genres

value,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
anime_id,,,,,,,,,,,,,,,,,,,,,
342,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6374,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6721,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9689,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
17693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20087,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
20543,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
27525,1,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
29995,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
random_user_ponderated_score = ponderated_score[ponderated_score.index.isin([random_user])]

In [32]:
random_user_ponderated_score

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
user_id,,,,,,,,,,,,,,,,,,,,,
63558,0.10581,0.061947,0.0,0.100423,0.0,0.007311,0.066179,0.005387,0.095421,0.011158,...,0.0,0.041554,0.0,0.017699,0.00808,0.067718,0.004617,0.005002,0.0,0.0


In [33]:
r = (
    random_user_ponderated_score.dot(random_animes_genres.T)
    .T
    .rename(columns = {random_user:"score"})
    .sort_values(by = "score", ascending = False)
    # .pipe(lambda df: pd.concat([df, random_animes_genres], axis = 1))
)

In [34]:
r

user_id,score
anime_id,
27525,0.401693
6374,0.257791
20543,0.216622
9689,0.167757
6721,0.105810
342,0.075029
20087,0.011158
17693,0.008850
29995,0.000000


In [35]:
(
    random_user_ponderated_score
    .melt()
    .sort_values("value", ascending = False)
)

,variable,value
0,Action,0.105810
3,Comedy,0.100423
8,Fantasy,0.095421
32,Shounen,0.081185
25,Romance,0.068873
38,Supernatural,0.067718
6,Drama,0.066179
27,School,0.065795
1,Adventure,0.061947
16,Magic,0.058484


In [36]:
i = 1
(
    random_animes_genres.loc[r.index[i]][random_animes_genres.loc[r.index[i]] != 0]
)

value
Action       1
Adventure    1
Sci-Fi       1
Shounen      1
Name: 6374, dtype: int64